In [827]:
import pandas as pd

# Resources
schools_file = "Resources/schools_complete.csv"
students_file = "Resources/students_complete.csv"

# Pull in resources
schools_csv = pd.read_csv(schools_file)
# schools_csv.head()

In [828]:
students_csv = pd.read_csv(students_file)
# students_csv.head()
# students_csv.columns

In [829]:
# District Summary ######################################################################

In [830]:
schools_with_students = pd.merge(schools_csv,students_csv, on=['school_name','school_name'])
# schools_with_students.head()

In [831]:
total_schools = schools_csv['school_name'].count()
# total_schools

In [832]:
# Rename columns so that they are differentiated
schools_with_students = schools_with_students.rename(columns={
    "school_name": "School Name", 
    "type": "School Type", 
    "budget": "Total School Budget",
    }
)
# schools_with_students.head()

In [833]:
# counts as string
total_students = schools_with_students['student_name'].count()
# '{:,}'.format(total_students)

In [834]:
total_budget = schools_csv['budget'].sum()
# '${:,.2f}'.format(total_budget)

In [835]:
average_math_score = schools_with_students['math_score'].mean()
# average_math_score
# average_math_score_rounded = round(average_math_score,6)
# average_math_score_rounded

average_reading_score = schools_with_students['reading_score'].mean()
# average_reading_score

passing_math_score_percentage = sum((schools_with_students['math_score']>=70)) / total_students * 100
# passing_math_score_percentage

passing_reading_score_percentage = sum((schools_with_students['reading_score']>=70)) / total_students * 100
# passing_reading_score_percentage

In [836]:
overall_passing_rate_percentage = ( sum((schools_with_students['math_score']>=70)) + sum((schools_with_students['reading_score']>=70)))
overall_passing_rate_percentage = overall_passing_rate_percentage / total_students
overall_passing_rate_percentage = overall_passing_rate_percentage / 2
overall_passing_rate_percentage = overall_passing_rate_percentage * 100
# overall_passing_rate_percentage

In [837]:
# schools_with_students.head()

In [838]:
# create dataframe for district summary output
district_summary = pd.DataFrame(
{"Total Schools" : [total_schools],
"Total Students" : ['{:,}'.format(total_students)],
"Total Budget" : ['${:,.2f}'.format(total_budget)],
"Average Math Score" : [round(average_math_score,2)], # 2 instead of 6 on homework assignment
"Average Reading Score" : [round(average_reading_score,2)], # 2 instead of 6 on homework assignment
"% Passing Math" : [round(passing_math_score_percentage,2)], # 2 instead of 6 on homework assignment
"% Passing Reading" : [round(passing_reading_score_percentage,2)], # 2 instead of 6 on homework assignment
"% Overall Passing Rate" : [round(overall_passing_rate_percentage,2)] # 2 instead of 6 on homework assignment
})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.39


In [839]:
# School Summary ######################################################################

In [840]:
# schools_with_students.columns

In [841]:
# Rename columns so that they are differentiated
schools_with_students = schools_with_students.rename(columns={
    "School Name": "school_name", 
    "School Type": "type", 
    "Total School Budget": "budget",
    "Student ID": "student_id"
    }
)
# schools_with_students.head()

In [842]:
# first add calculated columns required for further processing
schools_with_students['passing_math'] = schools_with_students['math_score'].apply(lambda x: 1 if x >= 70 else 0)
schools_with_students['passing_reading'] = schools_with_students['reading_score'].apply(lambda x: 1 if x >= 70 else 0)
# schools_with_students.head()

In [882]:
# aggregate columns for grouped values
aggregation = { 
     "budget": 'unique' ,
     "type": 'unique' ,
     "reading_score": 'mean' ,
     "student_id": 'count' ,
     "math_score": 'mean' ,
     "passing_math": 'sum',
     "passing_reading": 'sum',
}

school_summary = schools_with_students.groupby('school_name',as_index=False).agg(aggregation)
school_summary["per_student_budget"] = school_summary["budget"] / school_summary["student_id"]
school_summary["percentage_passing_reading"] = school_summary["passing_reading"] / school_summary["student_id"] * 100
school_summary["percentage_passing_math"] = school_summary["passing_math"] / school_summary["student_id"] * 100
school_summary["percentage_passing_overall"] = ( school_summary["percentage_passing_reading"] + 
                                                school_summary["percentage_passing_math"] ) / 2

# school_summary.head()
school_summary["budget"].to_frame()
school_summary.head()
test = school_summary["budget"]

In [844]:
# Top / Bottom Performing Schools (By Passing Rate) ######################################################################

In [845]:
school_top_bottom = school_summary

# rename columns
school_top_bottom = school_top_bottom.rename(columns={
    "school_name": "School Name", 
    "type": "School Type", 
    "student_id": "Total Students", 
    "budget": "Total School Budget", 
    "per_student_budget": "Per Student Budget", 
    "math_score": "Average Math Score", 
    "reading_score": "Average Reading Score", 
    "percentage_passing_math": "% Passing Math", 
    "percentage_passing_reading": "% Passing Reading", 
    "percentage_passing_overall": "% Overall Passing Rate"
    }
)
# school_top_bottom = school_top_bottom[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
school_top_bottom = school_top_bottom[[
    'School Name',
    'School Type',
    'Total Students',
    'Total School Budget',
    'Per Student Budget',
    'Average Math Score',
    'Average Reading Score', 
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]]

# group by school name
school_top_bottom.set_index('School Name',inplace=True)
school_top_bottom.groupby('School Name')

school_top_bottom.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,[District],4976,[3124928],[628.0],77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,[Charter],1858,[1081356],[582.0],83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,[District],2949,[1884411],[639.0],76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,[District],2739,[1763916],[644.0],77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,[Charter],1468,[917500],[625.0],83.351499,83.816757,93.392371,97.138965,95.265668


In [846]:
# Bottom Performing Schools (By Passing Rate) ######################################################################

In [847]:
# school_top = school_top_bottom
school_top = school_top_bottom.loc[school_top_bottom["% Overall Passing Rate"] > 95, :]
# sort descending 
school_top.sort_values(by=['% Overall Passing Rate'], ascending=False)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,[Charter],1858,[1081356],[582.0],83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,[Charter],1635,[1043130],[638.0],83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,[Charter],962,[585858],[609.0],83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,[Charter],1468,[917500],[625.0],83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,[Charter],2283,[1319574],[578.0],83.274201,83.989488,93.867718,96.539641,95.203679


In [848]:
# Top Performing Schools (By Passing Rate) ######################################################################

In [849]:
# school_top = school_top_bottom
school_top = school_top_bottom.loc[school_top_bottom["% Overall Passing Rate"] < 73.805, :]
# sort ascending 
school_top.sort_values(by=['% Overall Passing Rate'], ascending=True)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,[District],3999,[2547363],[637.0],76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,[District],2949,[1884411],[639.0],76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,[District],2917,[1910635],[655.0],76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,[District],4761,[3094650],[650.0],77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,[District],2739,[1763916],[644.0],77.102592,80.746258,68.309602,79.299014,73.804308


In [850]:
# Math Scores by Grade ######################################################################

In [851]:
ms_grade_9 = schools_with_students.loc[schools_with_students['grade'] == '9th',:][['school_name','math_score']]
ms_grade_9 = ms_grade_9.rename(columns={"math_score": "rs9"})
ms_grade_9.set_index('school_name')
ms_grade_9 = ms_grade_9.groupby('school_name')
# ms_grade_9.head()

ms_grade_9_sum = ms_grade_9.mean()
# ms_grade_9_sum

ms_grade_10 = schools_with_students.loc[schools_with_students['grade'] == '10th',:][['school_name','math_score']]
ms_grade_10 = ms_grade_10.rename(columns={"math_score": "rs10"})
ms_grade_10.set_index('school_name')
ms_grade_10 = ms_grade_10.groupby('school_name')
# ms_grade_10.head()

ms_grade_10_sum = ms_grade_10.mean()
# ms_grade_10_sum

ms_grade_11 = schools_with_students.loc[schools_with_students['grade'] == '11th',:][['school_name','math_score']]
ms_grade_11 = ms_grade_11.rename(columns={"math_score": "rs11"})
ms_grade_11.set_index('school_name')
ms_grade_11 = ms_grade_11.groupby('school_name')
# ms_grade_11.head()

ms_grade_11_sum = ms_grade_11.mean()
# ms_grade_11_sum

ms_grade_12 = schools_with_students.loc[schools_with_students['grade'] == '12th',:][['school_name','math_score']]
ms_grade_12 = ms_grade_12.rename(columns={"math_score": "rs12"})
ms_grade_12.set_index('school_name')
ms_grade_12 = ms_grade_12.groupby('school_name')
# ms_grade_12.head()

ms_grade_12_sum = ms_grade_12.mean()
# ms_grade_12_sum

math_scores_by_grade = ms_grade_9_sum
math_scores_by_grade['rs10'] = ms_grade_10_sum['rs10']
math_scores_by_grade['rs11'] = ms_grade_11_sum['rs11']
math_scores_by_grade['rs12'] = ms_grade_12_sum['rs12']
# math_scores_by_grade.head()

math_scores_by_grade = math_scores_by_grade.rename(columns={
    "rs9": "9th",
    "rs10": "10th",
    "rs11": "11th",
    "rs12": "12th"
})
math_scores_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [852]:
# Reading Score by Grade ######################################################################

In [853]:
ms_grade_9 = schools_with_students.loc[schools_with_students['grade'] == '9th',:][['school_name','reading_score']]
ms_grade_9 = ms_grade_9.rename(columns={"reading_score": "rs9"})
ms_grade_9.set_index('school_name')
ms_grade_9 = ms_grade_9.groupby('school_name')
# ms_grade_9.head()

ms_grade_9_sum = ms_grade_9.mean()
# ms_grade_9_sum

ms_grade_10 = schools_with_students.loc[schools_with_students['grade'] == '10th',:][['school_name','reading_score']]
ms_grade_10 = ms_grade_10.rename(columns={"reading_score": "rs10"})
ms_grade_10.set_index('school_name')
ms_grade_10 = ms_grade_10.groupby('school_name')
# ms_grade_10.head()

ms_grade_10_sum = ms_grade_10.mean()
# ms_grade_10_sum

ms_grade_11 = schools_with_students.loc[schools_with_students['grade'] == '11th',:][['school_name','reading_score']]
ms_grade_11 = ms_grade_11.rename(columns={"reading_score": "rs11"})
ms_grade_11.set_index('school_name')
ms_grade_11 = ms_grade_11.groupby('school_name')
# ms_grade_11.head()

ms_grade_11_sum = ms_grade_11.mean()
# ms_grade_11_sum

ms_grade_12 = schools_with_students.loc[schools_with_students['grade'] == '12th',:][['school_name','reading_score']]
ms_grade_12 = ms_grade_12.rename(columns={"reading_score": "rs12"})
ms_grade_12.set_index('school_name')
ms_grade_12 = ms_grade_12.groupby('school_name')
# ms_grade_12.head()

ms_grade_12_sum = ms_grade_12.mean()
# ms_grade_12_sum

reading_scores_by_grade = ms_grade_9_sum
reading_scores_by_grade['rs10'] = ms_grade_10_sum['rs10']
reading_scores_by_grade['rs11'] = ms_grade_11_sum['rs11']
reading_scores_by_grade['rs12'] = ms_grade_12_sum['rs12']
# reading_scores_by_grade.head()

reading_scores_by_grade = reading_scores_by_grade.rename(columns={
    "rs9": "9th",
    "rs10": "10th",
    "rs11": "11th",
    "rs12": "12th"
})
reading_scores_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [854]:
# Scores by School Spending ######################################################################

In [855]:
# bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

base_score_school_spending = school_summary[['school_name','per_student_budget','math_score','reading_score','percentage_passing_math','percentage_passing_reading','percentage_passing_overall']]
# base_score_school_spending.head()

In [856]:
# use the bins to create a new column
base_score_school_spending["spending_ranges"] = pd.cut(base_score_school_spending["per_student_budget"], spending_bins, labels=group_names, right=False)
# base_score_school_spending.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [857]:
score_school_spending = base_score_school_spending [['spending_ranges','math_score','reading_score','percentage_passing_math','percentage_passing_reading','percentage_passing_overall']]
# score_school_spending
score_school_spending = score_school_spending.rename(columns={
    "spending_ranges": "Spending Ranges (Per Student)"
})
aggregation = { 
     "math_score": 'mean' ,
     "reading_score": 'mean' ,
     "percentage_passing_math": 'mean' ,
     "percentage_passing_reading": 'mean' ,
     "percentage_passing_overall": 'mean' 
}

score_school_spending_final = score_school_spending.groupby('Spending Ranges (Per Student)',as_index=True).agg(aggregation)
score_school_spending_final = score_school_spending_final.rename(columns={
    "math_score": "Average Math Score",
    "reading_score": "Average Reading Score",
    "percentage_passing_math": "% Passing Math",
    "percentage_passing_reading": "% Passing Reading",
    "percentage_passing_overall": "% Overall Passing Rate"
})
score_school_spending_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [858]:
# Scores by School Size ######################################################################

In [861]:
# bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
base_score_school_size = school_summary[['school_name','student_id','math_score','reading_score','percentage_passing_math','percentage_passing_reading','percentage_passing_overall']]
# use the bins to create a new column
base_score_school_size["size_ranges"] = pd.cut(base_score_school_size["student_id"], size_bins, labels=group_names, right=False)
# base_score_school_size.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [860]:
score_school_size = base_score_school_size [['size_ranges','math_score','reading_score','percentage_passing_math','percentage_passing_reading','percentage_passing_overall']]
# score_school_size
score_school_size = score_school_size.rename(columns={
    "size_ranges": "School Size"
})
aggregation = { 
     "math_score": 'mean' ,
     "reading_score": 'mean' ,
     "percentage_passing_math": 'mean' ,
     "percentage_passing_reading": 'mean' ,
     "percentage_passing_overall": 'mean' 
}

score_school_size_final = score_school_size.groupby('School Size',as_index=True).agg(aggregation)
score_school_size_final = score_school_size_final.rename(columns={
    "math_score": "Average Math Score",
    "reading_score": "Average Reading Score",
    "percentage_passing_math": "% Passing Math",
    "percentage_passing_reading": "% Passing Reading",
    "percentage_passing_overall": "% Overall Passing Rate"
})
score_school_size_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [863]:
# Scores by School Type ######################################################################

In [870]:
score_school_type = schools_with_students[['type','math_score','reading_score','percentage_passing_math','percentage_passing_reading','percentage_passing_overall']]
# score_school_type
score_school_type = score_school_type.rename(columns={
    "type": "School Type"
})
# score_school_type
aggregation = { 
     "math_score": 'mean' ,
     "reading_score": 'mean' ,
     "percentage_passing_math": 'mean' ,
     "percentage_passing_reading": 'mean' ,
     "percentage_passing_overall": 'mean' 
}

score_school_type_final = score_school_type.groupby('School Type',as_index=True).agg(aggregation)
score_school_type_final = score_school_type_final.rename(columns={
    "math_score": "Average Math Score",
    "reading_score": "Average Reading Score",
    "percentage_passing_math": "% Passing Math",
    "percentage_passing_reading": "% Passing Reading",
    "percentage_passing_overall": "% Overall Passing Rate"
})
score_school_type_final

TypeError: unhashable type: 'numpy.ndarray'